In [1]:
using Fermionic
using SparseArrays
using LinearAlgebra
using DataFrames
using CSV

# Fermionic Dataset

### Complementary functions

filter_lvl picks only states corresponding to fully occupied levels from the full basis (which are the only non zero elements in the fundamental state). Remember we are simulating our system in the basis with fixed particle number, which scales as $\binom{d}{N}$ with $d$ the dimension and $N$ the particle number, instead of the full $2^d$ basis.

map_to_2n takes these levels to the full $2^n$ basis of n spines, in order to input the initialize function in qiskit. Here n is the number of levels (in a system with $d=8$, $n=4$ degenerate levels and mapps to a 4 qubits system).

In [2]:
function map_to_2n(estado, index, d)
    l = size(estado)[1]
    #lo puedo hacer sparse si prefieren, es mas lindo pero quizas no le gusta a initialize del qiskit
    estado2n = zeros(l,2^d)
    index2 = index[1:size(estado)[2]]
    for j in 1:l
        for (i,v) in enumerate(index2)
            estado2n[j,Int(v+1)]=estado[j,i]
        end
    end
    return estado2n    
end

function filter_lvl(b)
    bin, d = size(b)
    f = [sum([b[t,k]*b[t,k+1] for k in 1:2:d]) for t in 1:bin]
    return f
end
;

### Simulation
We will use the Fermionic.jl package in order to exactly simulate the Hamiltonian for each coupling. We will then save each state together with the coupling strength and a binary label, which divides superconductor states form non superconductor states. As a criterior for the phase transition, we chose BCS gap activation, but there are many other options.

In [3]:
d = 8 # This will be a system with 4 two-fold degenerated levels
nume = Int(d/2) # We consider half-occupation. That is, we have d/2 number of fermions

e0 = 1.0
gmin = 0.0
gmax = 5.0
step = 0.01

b, index = basis_m(d,nume)

epsilon = [e0*(i-d/4-1/2) for i in 1:d/2]
gc = 1/sum(1 ./abs.(2*epsilon)) # This is the critical coupling from BCS which determines the phase transition
epsilon = sort([epsilon; epsilon])

h0 = sum([epsilon[i]*(cdc(b,index,i,i) + cdc(b,index,i+1,i+1)) for i in 1:2:(Int(d)-1)]) # Diagonal term of the hamiltonian
hi = sum([sum([if i==j spzeros(binomial(d,nume), binomial(d,nume)) else -(cdc(b,index,j,i+1)*cdc(b,index,j+1,i)) end
                    for i in 1:2:(Int(d)-1)]) for j in 1:2:(Int(d)-1)]) # Two body interaction 

g = zeros(0)
estado = zeros(Int((gmax-gmin)/step+1),binomial(d,nume)) # This can be done as sparse too, changing zeros for spzeros
labels = zeros(0)

count = 1
for k in gmin:step:gmax # Iteration over coupling strenghts
    append!(g,round(k, digits=10))
    h = h0 - (round(k, digits=10))*hi # We create the Hamiltonian
    fundamental = eigvecs(Matrix(h))[:,1] # We take the first eigenvector
    fundamental = fundamental/sqrt(fundamental'*fundamental) # Normalization
    fundamental = [round(fundamental[i], digits=15) for i in 1:binomial(d,nume)] # We round to the specified precision
    estado[count,:] = fundamental
    if k>=gc # The superconductor label
        lab=1
    else
        lab=0
    end
    append!(labels,lab)
    count = count+1
end

In [4]:
gc

0.375

### Traduction to Qiskit and saving

We finally convert this state to the qiskit $2^n$ basis with $n$ the number of levels. In order to do that, we use the previously defined functions.

In [5]:
a = filter_lvl(b)
posiciones = findall(x->x==2, a)
e = estado[:,posiciones]
b2,i2=basis_m(Int(d/2),Int(nume/2))

estado2n=map_to_2n(e, i2, Int(d/2))

501×16 Array{Float64,2}:
 0.0  0.0  0.0  0.0          0.0  0.0         …  0.0  1.0       0.0  0.0  0.0
 0.0  0.0  0.0  1.46663e-5   0.0  0.00167501     0.0  0.99998   0.0  0.0  0.0
 0.0  0.0  0.0  5.89933e-5   0.0  0.00336677     0.0  0.999918  0.0  0.0  0.0
 0.0  0.0  0.0  0.000133465  0.0  0.00507533     0.0  0.999813  0.0  0.0  0.0
 0.0  0.0  0.0  0.000238553  0.0  0.00680073     0.0  0.999664  0.0  0.0  0.0
 0.0  0.0  0.0  0.000374715  0.0  0.00854301  …  0.0  0.999469  0.0  0.0  0.0
 0.0  0.0  0.0  0.000542392  0.0  0.0103022      0.0  0.999227  0.0  0.0  0.0
 0.0  0.0  0.0  0.000742009  0.0  0.0120782      0.0  0.998937  0.0  0.0  0.0
 0.0  0.0  0.0  0.000973971  0.0  0.0138711      0.0  0.998598  0.0  0.0  0.0
 0.0  0.0  0.0  0.00123866   0.0  0.0156808      0.0  0.998207  0.0  0.0  0.0
 0.0  0.0  0.0  0.00153645   0.0  0.0175072   …  0.0  0.997765  0.0  0.0  0.0
 0.0  0.0  0.0  0.00186767   0.0  0.0193503      0.0  0.99727   0.0  0.0  0.0
 0.0  0.0  0.0  0.00223263   0.0  0.021

Let's add coupling in the first column and labels in the last

In [6]:
estado_l = hcat(g,estado2n,labels)

501×18 Array{Float64,2}:
 0.0   0.0  0.0  0.0  0.0          0.0  …  0.0  1.0       0.0  0.0  0.0  0.0
 0.01  0.0  0.0  0.0  1.46663e-5   0.0     0.0  0.99998   0.0  0.0  0.0  0.0
 0.02  0.0  0.0  0.0  5.89933e-5   0.0     0.0  0.999918  0.0  0.0  0.0  0.0
 0.03  0.0  0.0  0.0  0.000133465  0.0     0.0  0.999813  0.0  0.0  0.0  0.0
 0.04  0.0  0.0  0.0  0.000238553  0.0     0.0  0.999664  0.0  0.0  0.0  0.0
 0.05  0.0  0.0  0.0  0.000374715  0.0  …  0.0  0.999469  0.0  0.0  0.0  0.0
 0.06  0.0  0.0  0.0  0.000542392  0.0     0.0  0.999227  0.0  0.0  0.0  0.0
 0.07  0.0  0.0  0.0  0.000742009  0.0     0.0  0.998937  0.0  0.0  0.0  0.0
 0.08  0.0  0.0  0.0  0.000973971  0.0     0.0  0.998598  0.0  0.0  0.0  0.0
 0.09  0.0  0.0  0.0  0.00123866   0.0     0.0  0.998207  0.0  0.0  0.0  0.0
 0.1   0.0  0.0  0.0  0.00153645   0.0  …  0.0  0.997765  0.0  0.0  0.0  0.0
 0.11  0.0  0.0  0.0  0.00186767   0.0     0.0  0.99727   0.0  0.0  0.0  0.0
 0.12  0.0  0.0  0.0  0.00223263   0.0     0.0  0.9

Let's save it as a csv:

In [7]:
header2n=String[]
push!(header2n,"g")
for i in 0:(2^Int(d/2)-1)
    binary_base = bitstring(i)[65-Int(d/2):64]
    push!(header2n,binary_base)
end
push!(header2n,"label")

CSV.write("fermionic_dataset_4.csv",  DataFrame(estado_l), header=header2n);

### Higer dimensions (only for superconductors)

Only for the superconducting system: We can exploit the symmetries of the problem to simulate higher dimensions in a more efficient way. Remember: this is not doable for general fermionic systems.

In [12]:
function indx(arr)
    l = length(arr)
    ind = spzeros(0)
    for i=1:l
        if arr[i] != 0
            ind = sparse([ind; i])
        end
    end
    return ind
end

function map_to_2n(estado, index, d)
    l = size(estado)[1]
    #lo puedo hacer sparse si prefieren, es mas lindo pero quizas no le gusta a initialize del qiskit
    estado2n = zeros(l,2^d)
    index2 = index[1:size(estado)[2]]
    for j in 1:l
        for (i,v) in enumerate(index2)
            estado2n[j,Int(v+1)]=estado[j,i]
        end
    end
    return estado2n    
end
;

In [13]:
# With symmetries

d=8 # d will be the number of levels now. So this is an 8 levels system.
nume = Int(d/2)

o = Op(d) # We use Fermionic for bulding the fermionic basis.

e0 = 1.0
epsilon = [2*e0*(i-d/2-1/2) for i in 1:d]
gc = 1/sum(1 ./abs.(epsilon))
h0 = spzeros(binomial(d,nume),binomial(d,nume))
bam, index = basis_m(d,nume)

for i in 1:binomial(d,nume)
    h0[i,i] = epsilon'*bam[i,:]
end

hi = spzeros(binomial(d,nume),binomial(d,nume))
for i in 1:binomial(d,nume)
    for j in 1:binomial(d,nume)
        if length(findall(in(indx(bam[i,:])), indx(bam[j,:]))) == nume - 1
            hi[i,j] = 1
        end
    end
end

gmin = 0.0
gmax = 5.0
step = 0.01

g = zeros(0)
estado = zeros(Int((gmax-gmin)/step+1),binomial(d,nume)) # This can be done as sparse too, changing zeros for spzeros
labels = zeros(0)

count = 1
for k in gmin:step:gmax 
    append!(g,round(k, digits=10))
    h = h0 - k*hi
    fundamental = eigvecs(Matrix(h))[:,1]
    fundamental = fundamental/sqrt(fundamental'*fundamental) # Normalization
    fundamental = [round(fundamental[i], digits=15) for i in 1:binomial(d,nume)] # We round to the specified precision
    estado[count,:] = fundamental
    if k>=gc # The superconductor label
        lab=1
    else
        lab=0
    end
    append!(labels,lab)
    count = count+1
end
estado2n=map_to_2n(estado, index, d)
estado_l = hcat(g,estado2n,labels);

In [14]:
header2n=String[]
push!(header2n,"g")
for i in 0:(2^Int(d)-1)
    binary_base = bitstring(i)[65-Int(d):64]
    push!(header2n,binary_base)
end
push!(header2n,"label")

CSV.write("fermionic_dataset_8.csv",  DataFrame(estado_l), header=header2n);